## Homework 08: The Cycle of Twelve
##### By: Kevin Liu

In [1]:
import numpy as np
import scipy.stats as stats
import scipy.optimize as optimize
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import re
import os

### 1. Solve by Maximum Likelihood

Given the RNA-seq data and known $\sigma$, we will use maximum likelihood to solve for the unknown phases $\phi$.

We will first define a series of functions to read in the data, calculate the negative log likelihood of $y_t = b + a * sin( 2\pi\omega (t + \phi))$, identify the parameters that maximize the log likelihood, and display the maximum likelihood estimates $\hat{a}$, $\hat{b}$, and $\hat{\phi}$ for each gene.

In [2]:
def read_data(datafile):
    """
    Adapted from moriarty.py.
    Reads in the data file and returns all data.
    
    N         : number of experiments (columns in the table).
    G         : number of genes (rows in the table).
    X[i]      : array of time points, in hrs, for the N experiments.
    S_true[i] : array of sigmas for the experiments.
    Y[i][t]   : GxN: observed tpm for gene i, time point t.
    """
    with open(datafile) as f:
        # First header line gives us the time points
        fields = f.readline().split()
        X = []
        for s in fields:
            match = re.search(r"^(\d+)hr", s)
            X.append(int(match.group(1)))
        X = np.array(X)
        N = len(X)

        # Second header line gives us "gene" followed by +=SD's
        fields = f.readline().split()
        S_true = np.zeros(N)
        for i, s in enumerate(fields[1:]):
            match = re.search(r"^\+-(\d+)", s)
            S_true[i] = float(match.group(1))

        # Third header line is just ------ stuff
        f.readline()

        # Remaining lines are data
        genenames = []
        Y = []
        for line in f.readlines():
            fields = line.split()
            genenames.append(fields[0])
            Y.append(np.array([float(s) for s in fields[1:]]))
        G = len(Y)
        
    return genenames, G, X, S_true, Y


def calc_y_hat(a, b, phi, timePoint):
    """
    Calculates point estimate y_hat of gene expression.
    """
    return b + a*np.sin(2*np.pi*(1/24)*(timePoint+phi))


def nll(p, data, sigmas, timePoints):
    """
    Calculates the negative log-likelihood given model parameters, data, and respective sigmas and timepoints.
    """
    residuals = [x - calc_y_hat(*p, timePoint) for x, timePoint in zip(data, timePoints)]
    nll = -sum([stats.norm.logpdf(residual, 0, sigma) for residual, sigma in zip(residuals, sigmas)])
             
    return nll


def maxll(data, sigmas, timePoints, n_runs):
    """
    Fits the data with sigmas and timepoints to a maximum likelihood model by minimizing the negative 
    log-likelihood.
    Returns the model parameters per gene and negative log-likelihoods.
    """
    data_arr = np.asarray(data)
    opt_p_lst = []
    opt_f_lst = []
    for i in range(data_arr.shape[0]):
        opt_f = -1
        opt_p = -1
        for j in range(n_runs):
            x0 = np.array([np.random.uniform(0, 100), np.random.uniform(0, 100), np.random.uniform(0, 24)])
            bnd = [(0.0, None), (0.0, None), (None, None)]
            result = optimize.minimize(nll, x0, (data_arr[i], sigmas, timePoints), bounds = bnd)

            if result.fun < opt_f or opt_f == -1:
                result.x[2] = result.x[2]%24
                opt_f = result.fun
                opt_p = result.x

        opt_p_lst.append(opt_p)
        opt_f_lst.append(opt_f)

    return opt_p_lst, opt_f_lst


def print_p(p, genenames, G):
    """
    Prints out the model parameters per gene.
    """
    print("{0:12s} {1:>6s} {2:>6s} {3:>6s}".format("genename", "a", "b", "p"))
    print("{0:12s} {1:6s} {2:6s} {3:6s}".format("-"*12, "-"*6, "-"*6, "-"*6))
    for g in range(G):
        print("{0:12s} {1:6.2f} {2:6.2f} {3:6.2f}".format(genenames[g], p[g][0], p[g][1], p[g][2]))
    print()

After defining our functions, we can finally read in the data, calculate the optimal parameters for our gene expression data using our maximum likelihood model, and display the estimated model parameters $\hat{a}$, $\hat{b}$, and $\hat{\phi}$ for each gene.

In [3]:
geneNames, G, timePoints, sigmas, data = read_data("w08-data.tbl")

ML_p_lst, ML_nll_lst = maxll(data, sigmas, timePoints, n_runs = 10)
    
print("Maximum likelihood estimates:")
print_p(ML_p_lst, geneNames, G)

Maximum likelihood estimates:
genename          a      b      p
------------ ------ ------ ------
anise         22.42  39.99   0.45
kiwi          31.84  48.84  16.26
carrot        24.23  47.58  21.41
grape         32.33  44.90  20.42
tangerine     22.15  42.76   2.78
melon         19.10  51.26   6.69
clementine    27.60  48.02  10.16
spinach       18.40  49.73  14.27
beet          30.61  40.85   7.42
huckleberry   21.58  45.99  11.47
lentil        17.32  53.06   3.48
cauliflower   27.48  39.12  17.95



As displayed above, we have estimated the $\hat{a}$, $\hat{b}$, and $\hat{\phi}$ for each gene using maximum likelihood.

### 2. Compare Solutions

Now that we have obtained our estimated parameters using a maximum likelihood model, we will compare how well our maximum likelihood model performs relative to Moriaty's harmonic regression model in fitting a line to the gene expression data.

We will first run Moriarty's Python script ```moriarty.py``` and read in the estimated parameters. Then, we will use the resulting estimated parameters to calculate the associated negative log-likelihoods and use it to calculate a likelihood ratio relative to our maximum likelihood model fit.

In [4]:
m_lines = os.popen("./moriarty.py w08-data.tbl").readlines()[2:]

OLS_p_lst = []
for line in m_lines: 
    line = line.strip("\n").split()
    line = [float(i) for i in line[1:]]
    line[0], line[1] = line[1], line[0]
    OLS_p_lst.append(line)

OLS_nll_lst = []
for i, p in enumerate(OLS_p_lst):
    OLS_nll = nll(p, data[i], sigmas, timePoints)
    OLS_nll_lst.append(OLS_nll)
    
print("Maximum likelihood estimates:")
print_p(ML_p_lst, geneNames, G)
print("Moriarty's harmonic regression estimates:")
print_p(OLS_p_lst, geneNames, G)
ML_tot_nll = sum(ML_nll_lst)
OLS_tot_nll = sum(OLS_nll_lst)
print("Maximum likelihood total negative log-likelihood: " + str(np.round(ML_tot_nll, 2)))
print("Moriarty's harmonic regression total negative log-likelihood: " + str(np.round(OLS_tot_nll, 2)))
print("Our maximum likelihood fit parameters are {:.3} times as likely as Moriarty's harmonic regression \
parameters.".format(np.exp(-ML_tot_nll)/np.exp(-OLS_tot_nll)))

Maximum likelihood estimates:
genename          a      b      p
------------ ------ ------ ------
anise         22.42  39.99   0.45
kiwi          31.84  48.84  16.26
carrot        24.23  47.58  21.41
grape         32.33  44.90  20.42
tangerine     22.15  42.76   2.78
melon         19.10  51.26   6.69
clementine    27.60  48.02  10.16
spinach       18.40  49.73  14.27
beet          30.61  40.85   7.42
huckleberry   21.58  45.99  11.47
lentil        17.32  53.06   3.48
cauliflower   27.48  39.12  17.95

Moriarty's harmonic regression estimates:
genename          a      b      p
------------ ------ ------ ------
anise         15.67  48.56   0.61
kiwi          33.46  52.70  15.58
carrot        26.08  46.91  21.33
grape         30.17  41.68  20.21
tangerine     22.96  47.54   2.60
melon         27.44  46.48   8.16
clementine    18.32  42.09   9.36
spinach       14.39  51.88  14.26
beet          35.66  43.13   6.48
huckleberry   28.94  45.05  10.85
lentil        18.14  56.20   0.85
cauliflow

Based on the above results, we can conclude that our maximum likelihood model is 1.79e+58 times as likely as Moriarty's harmonic regression model. If Moriarty offered us even odds on his bet, we should take it since our maximum likelihood model has a significant edge over that of Moriarty's harmonic regression model.

### 3. Plot the Fits

To examine the model fits of both models, we will visualize the gene expression data using our respective model parameters and display how Moriarty's model is less likely than our maximum likelihood model.

In [ ]:
x_scale = np.arange(0, 24, 0.1)

fig, axs = plt.subplots(3, 4, figsize = (20, 10))
for i, ax in enumerate(axs.flat):
    ax.plot(x_scale, [calc_y_hat(*ML_p_lst[i], x) for x in x_scale], color="tab:blue", linewidth=2, alpha=0.75)
    ax.plot(x_scale, [calc_y_hat(*OLS_p_lst[i], x) for x in x_scale], color="tab:orange", linewidth=2, alpha=0.75)
    for j, x in enumerate(data[i]):
        ax.scatter(timePoints[j], x, c={2:"g", 5:"y", 20:"r"}[sigmas[j]], s=50)
    ax.set_title(geneNames[i])
    ax.set_xlabel("Time (Hours)")
    ax.set_ylabel("Expression Level (TPM)")

lgnd = [Line2D([0], [0], color="tab:blue", lw=2, label="Maximum Likelihood"),
        Line2D([0], [0], color="tab:orange", lw=2, label="Harmonic Regression"),
        Line2D([0], [0], marker="o", color="w", label="$\sigma$ = 2", markerfacecolor="g", markersize=10),
        Line2D([0], [0], marker="o", color="w", label="$\sigma$ = 5", markerfacecolor="y", markersize=10),
        Line2D([0], [0], marker="o", color="w", label="$\sigma$ = 20", markerfacecolor="r", markersize=10)]

fig.legend(handles=lgnd, bbox_to_anchor=(0.5, -0.01))
plt.tight_layout()

Based on the above plots, it is apparent that our maximum likelihood model fit line is closer to the data points with lower vairance (i.e., the green and yellow data points) and is less influenced by those with high variance when compared to that of Moriarty's harmonic regression model.

Given the results of Moriarty's harmonic regression model, we can conclude that the number of standard deviations contained within the residual is of greater significance in fitting the data than the size of the residuals.